In [1]:
#Instalaties die nodig zijn
# %pip install httpx
# %pip install selenium
# %pip install yfinance --upgrade --no-cache-dir
# %pip install nltk

In [2]:
# !pip install git+https://github.com/psf/requests-html.git#egg=requests-html
%reset
import httpx
import re
import requests
import codecs
import time
import nltk

import threading
import pandas as pd

from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from sklearn.preprocessing import MinMaxScaler


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
          }

YAHOO_CONSENT_COOKIES = None

DATA_LOCATION = 'Datasets/'

HOMEPAGE_URL = 'https://www.investopedia.com/markets-news-4427704'

VUSA_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500-companies'
SP_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500'
DATAHUB_URL = 'https://datahub.io'


YAHOO_URL = 'https://finance.yahoo.com/'

nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'])

[nltk_data] Downloading package punkt to C:\Users\Daan Monster PC
[nltk_data]     2.0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Daan Monster PC
[nltk_data]     2.0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Daan Monster PC
[nltk_data]     2.0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Daan Monster PC
[nltk_data]     2.0\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Daan Monster
[nltk_data]     PC 2.0\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu') 

In [5]:
def get_yahoo_cookies():
    yahoo_cookie_driver = webdriver.Chrome(options=options)
    yahoo_cookie_driver.get(YAHOO_URL)
    time.sleep(2)
    yahoo_cookie_driver.find_element(By.CLASS_NAME, 'reject-all').click()
    time.sleep(2)
    all_cookies=yahoo_cookie_driver.get_cookies();
    cookies_dict = {}
    for cookie in all_cookies:
        cookies_dict[cookie['name']] = cookie['value']
    cookies = [{'name': cookie[0], 'value': cookie[1]} for cookie in cookies_dict.items()]
    return cookies

In [6]:
YAHOO_CONSENT_COOKIES = get_yahoo_cookies()

In [7]:
def get_response(url, cookies={}, headers=HEADERS):
    r = httpx.get(url, cookies=cookies, headers=headers, follow_redirects=True)
    soup = bs(r.content)
    return soup

In [8]:
soup = get_response(HOMEPAGE_URL, HEADERS)

In [9]:
def get_card_titles():
    titles = []
    a_tags = soup.find_all('a', class_='card')
    for i in range(len(a_tags)):
        titles.append(a_tags[i].span.text)
    return titles

def get_card_urls():
    urls = []
    a_tags = soup.find_all('a', class_='card')
    for i in range(len(a_tags)):
        urls.append(a_tags[i]['href'])
    return urls

In [10]:
titles = get_card_titles()
urls = get_card_urls()

In [11]:
investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
display(investopedia_df)

,url,title
0,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Miner Cameco Gets Boost ...
1,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Kroger in Focus as Merge...
2,https://www.investopedia.com/dow-jones-today-0...,"Dow Jones Today: Stocks Slip as Dollar Surges,..."
3,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens
4,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week
...,...,...
60,https://www.investopedia.com/organized-labor-w...,Organized Labor’s Wins in 2023 May Spur More S...
61,https://www.investopedia.com/midday-movers-amd...,Midday Movers: AMD Higher on Graphics Card Rep...
62,https://www.investopedia.com/investors-electio...,Investors' Election Year Worries Could Be Over...
63,https://www.investopedia.com/dow-jones-today-1...,"Markets News, Dec. 28, 2023: S&P 500 Edges Tow..."


In [12]:
def clean_text(text):
    return re.sub(r'[^\w\s]', '', text)

In [13]:
def get_url_content(urls):
    articles = []
    raw_articles = []
    for url in urls:
        soup = get_response(url)
        paragraphs = soup.find('div', class_='article-content').find_all('p')
        article = ''.join([p.get_text(separator=' ') for p in paragraphs])
        
        raw_articles.append(paragraphs)
        articles.append(clean_text(article))
    return articles, raw_articles

def get_symbols(raw_articles, href_ref):
    symbol_list = []
    for p_list in raw_articles:
        article_symbols = []
        for p in p_list:
            symbols = [a_tag.text for a_tag in p.find_all('a') if re.search(href_ref, a_tag['href'])]
            if symbols:
                for symbol in symbols:
                    article_symbols.append(symbol)
        symbol_list.append(article_symbols)
    return symbol_list

In [14]:
articles, raw_articles = get_url_content(investopedia_df['url'])

In [15]:
investopedia_df['article'] = articles

In [16]:
symbols = get_symbols(raw_articles, 'widgetsymbol')

In [17]:
investopedia_df['symbols'] = symbols

In [18]:
display(investopedia_df)

,url,title,article,symbols
0,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Miner Cameco Gets Boost ...,\nHere is a look at a chart tied to Camecos re...,[CCJ]
1,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Kroger in Focus as Merge...,\nHere is a look at a chart tied to recent ne...,"[KR, ACI]"
2,https://www.investopedia.com/dow-jones-today-0...,"Dow Jones Today: Stocks Slip as Dollar Surges,...",\nStocks were mixed in early trading Tuesday a...,"[GS, MS, AAPL, MASI, SNPS, ANSS]"
3,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens,\nGoldman Sachs and Morgan Stanley report earn...,"[GS, MS, TSLA, BA, SPR, MSFT, KR, ACI]"
4,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week,\nMarkets are closed on Monday Jan 15 for Mart...,"[MS, GS, MS, GS, SCHW, DFS, MTB, FITB, STT, WF..."
...,...,...,...,...
60,https://www.investopedia.com/organized-labor-w...,Organized Labor’s Wins in 2023 May Spur More S...,\nOrganized labor had its most active year in ...,"[T, BA, GM, F, STLA, UPS, WMT, AMZN]"
61,https://www.investopedia.com/midday-movers-amd...,Midday Movers: AMD Higher on Graphics Card Rep...,\nUS equities traded marginally higher at midd...,"[AMD, MTCH, NKE, BA, PANW]"
62,https://www.investopedia.com/investors-electio...,Investors' Election Year Worries Could Be Over...,\nAs the 2024 US presidential election nears m...,[]
63,https://www.investopedia.com/dow-jones-today-1...,"Markets News, Dec. 28, 2023: S&P 500 Edges Tow...",\n UpdateDec 29 2023 For todays live markets ...,"[XLK, PENN, AAPL, MASI, APA, EOG, COP, BA, SPR]"


In [19]:
def get_datahub_csv(URL, filename):
    soup = get_response(URL)
    download_url = soup.find_all('table')[1].find_all('a')[1]['href']
    download_url = DATAHUB_URL + download_url
    with open(DATA_LOCATION + filename, 'wb') as f:
        f.write(requests.get(download_url).content)

In [20]:
get_datahub_csv(VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
get_datahub_csv(SP_DOWNLOAD_URL, 'time_data.csv')

In [21]:
vusa_df = pd.read_csv(DATA_LOCATION + 'vusa_holdings.csv')
symbols_in_vusa = vusa_df['Symbol'].tolist()

In [22]:
investopedia_df['appears_in_vusa'] = investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))
investopedia_df = investopedia_df[investopedia_df['appears_in_vusa']]

In [23]:
def get_article_links(driver):
    links = driver.find_elements(By.TAG_NAME, 'a')
    return links

In [24]:
def get_yahoo_news_articles_df():
    yahoo_news_driver = webdriver.Chrome(options=options)

    yahoo_news_driver.get(YAHOO_URL)
    time.sleep(1)
    yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
    actions = ActionChains(yahoo_news_driver)
    for i in range(1500):
        actions.scroll_by_amount(0, 500)
    actions.perform()
    actions.reset_actions()

    links = get_article_links(yahoo_news_driver)
    df = pd.DataFrame({'title': [link.text for link in links], 'url': [link.get_attribute('href') for link in links], 'classes': [link.get_attribute('class') for link in links]})
    yahoo_news_driver.quit()
    df = df[df['url'] \
            .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
            .drop_duplicates(subset='url', keep='last')\
            .reset_index(drop=True)
    return df

In [25]:
yahoo_news_df = get_yahoo_news_articles_df()

In [26]:
display(yahoo_news_df)

,title,url,classes
0,Stock market today: US stocks open lower to st...,https://finance.yahoo.com/news/stock-market-to...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...
1,Iowa Republicans focused on the economy and th...,https://finance.yahoo.com/news/iowa-republican...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...
2,"PepsiCo CEO on leadership, ultra-processed foo...",https://finance.yahoo.com/news/pepsico-ceo-on-...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
3,'Sheer panic' from European leaders on potenti...,https://finance.yahoo.com/news/sheer-panic-fro...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
4,Goldman posts 51% gain in last quarter of Davi...,https://finance.yahoo.com/news/goldman-posts-5...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
...,...,...,...
183,Is Nike Stock a No-Brainer Buy for 2024?,https://finance.yahoo.com/news/nike-stock-no-b...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
184,Investment Banker Turned Oil Tycoon Takes Cana...,https://finance.yahoo.com/news/investment-bank...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
185,Musk wants 25% voting control at Tesla before ...,https://finance.yahoo.com/news/musk-seeks-25-v...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...
186,Elon Musk building AI outside of Tesla 'would ...,https://finance.yahoo.com/news/elon-musk-build...,js-content-viewer wafer-caas Td(n) D(ib) Va(t)...


In [27]:
def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

In [28]:
def scrape_articles(urls):
    articles = []
    raw_articles = []
    yahoo_driver = webdriver.Chrome(options=options)
    yahoo_driver.get(YAHOO_URL)
    yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

    for index, url in enumerate(urls):
        yahoo_driver.get(url)
        try:
            yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
        except:
            pass
        article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
        article_by_paragraph = [p.text for p in article_p_elements if p.text!='']
        
        raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
        article = " ".join(article_by_paragraph)
        clean_article = clean_text(article)
        articles.append(clean_article)
        raw_articles.append(raw_article)
        progress_bar(index+1, len(urls))
    
        
    yahoo_driver.quit()
    
    return articles, raw_articles

In [29]:
yahoo_news_articles, raw_yahoo_news_articles = scrape_articles(yahoo_news_df['url'])

Progress: [------------------->] 100%


In [30]:
yahoo_news_df['article'] = yahoo_news_articles

In [31]:
yahoo_news_df

,title,url,classes,article
0,Stock market today: US stocks open lower to st...,https://finance.yahoo.com/news/stock-market-to...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...,US stocks retreated on Tuesday with investors ...
1,Iowa Republicans focused on the economy and th...,https://finance.yahoo.com/news/iowa-republican...,js-content-viewer Fw(b) Fz(18px) Lh(23px) Line...,There were two main concerns from voters that ...
2,"PepsiCo CEO on leadership, ultra-processed foo...",https://finance.yahoo.com/news/pepsico-ceo-on-...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,PepsiCo PEP chairman and CEO Ramon Laguarta is...
3,'Sheer panic' from European leaders on potenti...,https://finance.yahoo.com/news/sheer-panic-fro...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,Donald Trump got one step closer to a second p...
4,Goldman posts 51% gain in last quarter of Davi...,https://finance.yahoo.com/news/goldman-posts-5...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,Goldman Sachs GS in 2023 recorded the lowest p...
...,...,...,...,...
183,Is Nike Stock a No-Brainer Buy for 2024?,https://finance.yahoo.com/news/nike-stock-no-b...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,Nike NYSE NKE was among the worst performers o...
184,Investment Banker Turned Oil Tycoon Takes Cana...,https://finance.yahoo.com/news/investment-bank...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,Bloomberg In just over six years former inves...
185,Musk wants 25% voting control at Tesla before ...,https://finance.yahoo.com/news/musk-seeks-25-v...,js-content-viewer wafer-caas Fw(b) Fz(18px) Lh...,By Chandni Shah and Hyunjoo Jin Reuters Tesla ...
186,Elon Musk building AI outside of Tesla 'would ...,https://finance.yahoo.com/news/elon-musk-build...,js-content-viewer wafer-caas Td(n) D(ib) Va(t)...,Tesla TSLA CEO Elon Musk warned Monday that he...


In [32]:
def combine_dfs(investopedia_df, yahoo_df):
    df1 = investopedia_df.drop(['symbols', 'appears_in_vusa'], axis=1)
    df2 = yahoo_df.drop('classes', axis=1)
    article_df = pd.concat([df1, df2])
    article_df = article_df.reset_index(drop=True)
    return article_df

In [33]:
df = combine_dfs(investopedia_df, yahoo_news_df)

In [34]:
def clean_and_tokenize(articles):
    lemmatizer = WordNetLemmatizer()
    
    tokenized_articles = []
    for article in articles:
        tokenized_article = nltk.word_tokenize(article)
        tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
        tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]
        
        tokenized_articles.append(tokenized_article)
        
    return tokenized_articles

In [35]:
df['tokenized_article'] = clean_and_tokenize(df['article'])

In [36]:
df

,url,title,article,tokenized_article
0,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Kroger in Focus as Merge...,\nHere is a look at a chart tied to recent ne...,"[Here, look, chart, tied, recent, news, import..."
1,https://www.investopedia.com/dow-jones-today-0...,"Dow Jones Today: Stocks Slip as Dollar Surges,...",\nStocks were mixed in early trading Tuesday a...,"[Stocks, mixed, early, trading, Tuesday, Treas..."
2,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens,\nGoldman Sachs and Morgan Stanley report earn...,"[Goldman, Sachs, Morgan, Stanley, report, earn..."
3,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week,\nMarkets are closed on Monday Jan 15 for Mart...,"[Markets, closed, Monday, Jan, 15, Martin, Lut..."
4,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Airline Stocks...,\nUS equities were mixed as earnings season go...,"[US, equity, mixed, earnings, season, got, und..."
...,...,...,...,...
243,https://finance.yahoo.com/news/nike-stock-no-b...,Is Nike Stock a No-Brainer Buy for 2024?,Nike NYSE NKE was among the worst performers o...,"[Nike, NYSE, NKE, among, worst, performer, Dow..."
244,https://finance.yahoo.com/news/investment-bank...,Investment Banker Turned Oil Tycoon Takes Cana...,Bloomberg In just over six years former inves...,"[Bloomberg, In, six, year, former, investment,..."
245,https://finance.yahoo.com/news/musk-seeks-25-v...,Musk wants 25% voting control at Tesla before ...,By Chandni Shah and Hyunjoo Jin Reuters Tesla ...,"[By, Chandni, Shah, Hyunjoo, Jin, Reuters, Tes..."
246,https://finance.yahoo.com/news/elon-musk-build...,Elon Musk building AI outside of Tesla 'would ...,Tesla TSLA CEO Elon Musk warned Monday that he...,"[Tesla, TSLA, CEO, Elon, Musk, warned, Monday,..."


In [37]:
analyzer = SentimentIntensityAnalyzer()
scaler = MinMaxScaler()
df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in df['article']]
df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in df['article']]

In [38]:
df

,url,title,article,tokenized_article,positivity_score,negativity_score
0,https://www.investopedia.com/stock-chart-to-wa...,Stock Chart to Watch: Kroger in Focus as Merge...,\nHere is a look at a chart tied to recent ne...,"[Here, look, chart, tied, recent, news, import...",0.057,0.070
1,https://www.investopedia.com/dow-jones-today-0...,"Dow Jones Today: Stocks Slip as Dollar Surges,...",\nStocks were mixed in early trading Tuesday a...,"[Stocks, mixed, early, trading, Tuesday, Treas...",0.101,0.048
2,https://www.investopedia.com/5-things-to-know-...,5 Things to Know Before The Stock Market Opens,\nGoldman Sachs and Morgan Stanley report earn...,"[Goldman, Sachs, Morgan, Stanley, report, earn...",0.091,0.010
3,https://www.investopedia.com/what-to-expect-in...,What To Expect in the Markets This Week,\nMarkets are closed on Monday Jan 15 for Mart...,"[Markets, closed, Monday, Jan, 15, Martin, Lut...",0.077,0.045
4,https://www.investopedia.com/s-and-p-500-gains...,S&P 500 Gains and Losses Today: Airline Stocks...,\nUS equities were mixed as earnings season go...,"[US, equity, mixed, earnings, season, got, und...",0.118,0.086
...,...,...,...,...,...,...
243,https://finance.yahoo.com/news/nike-stock-no-b...,Is Nike Stock a No-Brainer Buy for 2024?,Nike NYSE NKE was among the worst performers o...,"[Nike, NYSE, NKE, among, worst, performer, Dow...",0.135,0.075
244,https://finance.yahoo.com/news/investment-bank...,Investment Banker Turned Oil Tycoon Takes Cana...,Bloomberg In just over six years former inves...,"[Bloomberg, In, six, year, former, investment,...",0.127,0.037
245,https://finance.yahoo.com/news/musk-seeks-25-v...,Musk wants 25% voting control at Tesla before ...,By Chandni Shah and Hyunjoo Jin Reuters Tesla ...,"[By, Chandni, Shah, Hyunjoo, Jin, Reuters, Tes...",0.124,0.035
246,https://finance.yahoo.com/news/elon-musk-build...,Elon Musk building AI outside of Tesla 'would ...,Tesla TSLA CEO Elon Musk warned Monday that he...,"[Tesla, TSLA, CEO, Elon, Musk, warned, Monday,...",0.100,0.088


# Yahoo Finance dataframe


In [39]:
#Imports
import yfinance as yf

In [40]:
#Dataframe maken met alle symbolen die voorkomen in de VUSA ETF
# vusa_holdings dataset inlezen.
file_path = 'datasets/vusa_holdings.csv'
df = pd.read_csv(file_path)

# Symbolen extraheren uit de eerste kolom.
symbol_list = df.iloc[:, 0].tolist()

#Vusa toevoegen
#Vusa heeft geen waarden dus niet meenenem
#symbol_list += ['VUSA']

#print(symbol_list)

In [41]:
# Data downloaden vanaf yahoo finance met behulp van API. 
#Hiervan alleen de tickers die voorkomen in VUSA.
download = yf.download(symbol_list, group_by="ticker")

[*********************100%%**********************]  505 of 505 completed

27 Failed downloads:
['FRC', 'ANTM', 'FB', 'VIAC', 'FBHS', 'BLL', 'CERN', 'ATVI', 'DISCA', 'BRK.B', 'INFO', 'PBCT', 'RE', 'DISCK', 'SIVB', 'NLSN', 'NLOK', 'PKI', 'DRE', 'ABC', 'FISV', 'KSU', 'CTXS', 'XLNX', 'WLTW', 'TWTR']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1925-02-08 -> 2024-01-16)')


In [42]:
#Kopie maken zodat de downloaden niet telkens uitgevoerd hoeft te worden.
data = download.copy()

In [43]:
#Tickers waarvan geen data kon worden opgehaald verwijderen uit dataframe.
filtered_data = data.dropna(axis=1, how='all')
filtered_data.columns = filtered_data.columns.remove_unused_levels()
filtered_data.shape

(15616, 2868)

In [44]:
#Checken of de tickers goed zijn verwijdert
remaining_tickers = list(filtered_data.columns.levels[0])
missing_tickers = list(set(symbol_list) - set(remaining_tickers))

#ontbrekende tickers en het aantal
print(missing_tickers, len(missing_tickers))

['NLSN', 'FB', 'RE', 'WLTW', 'FISV', 'PBCT', 'ATVI', 'CERN', 'DISCA', 'ANTM', 'BRK.B', 'FBHS', 'INFO', 'DRE', 'DISCK', 'KSU', 'CTXS', 'NLOK', 'BF.B', 'TWTR', 'ABC', 'SIVB', 'VIAC', 'BLL', 'XLNX', 'FRC', 'PKI'] 27


In [45]:
#Dataframe tonen
display(filtered_data, filtered_data.shape)

FLT                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
1962-01-02         NaN         NaN         NaN         NaN         NaN   
1962-01-03         NaN         NaN         NaN         NaN         NaN   
1962-01-04         NaN         NaN         NaN         NaN         NaN   
1962-01-05         NaN         NaN         NaN         NaN         NaN   
1962-01-08         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2024-01-09  282.600006  285.200012  282.049988  283.690002  283.690002   
2024-01-10  283.670013  283.899994  279.820007  280.260010  280.260010   
2024-01-11  280.989990  281.320007  277.559998  279.480011  279.480011   
2024-01-12  282.070007  284.000000  280.250000  283.989990  283.989990   
2024-01-16  282.200012  284.119995  280.040009  282.209991  282.209991   

                           HSIC                                   ...  \
              Volume       Open       High        Low      Close  ...   
Date                                                              ...   
1962-01-02       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-03       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-04       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-05       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-08       NaN        NaN        NaN        NaN        NaN  ...   
...              ...        ...        ...        ...        ...  ...   
2024-01-09  263900.0  75.160004  76.050003  74.870003  75.059998  ...   
2024-01-10  241500.0  74.970001  75.980003  74.580002  74.730003  ...   
2024-01-11  238200.0  74.650002  75.040001  73.220001  73.949997  ...   
2024-01-12  258800.0  74.180000  74.699997  73.559998  73.760002  ...   
2024-01-16   71026.0        NaN        NaN        NaN        NaN  ...   

                   ROK                                          AEP  \
                   Low       Close   Adj Close    Volume       Open   
Date                                                                  
1962-01-02         NaN         NaN         NaN       NaN   0.000000   
1962-01-03         NaN         NaN         NaN       NaN   0.000000   
1962-01-04         NaN         NaN         NaN       NaN   0.000000   
1962-01-05         NaN         NaN         NaN       NaN   0.000000   
1962-01-08         NaN         NaN         NaN       NaN   0.000000   
...                ...         ...         ...       ...        ...   
2024-01-09  302.359985  302.970001  302.970001  578500.0  84.019997   
2024-01-10  301.170013  302.070007  302.070007  689200.0  83.870003   
2024-01-11  299.769989  305.100006  305.100006  703400.0  81.389999   
2024-01-12  302.290009  304.589996  304.589996  455200.0  80.860001   
2024-01-16  299.130005  300.412506  300.412506   90322.0  81.000000   

                                                                 
                 High        Low      Close  Adj Close   Volume  
Date                                                             
1962-01-02  35.125000  34.312500  34.312500   0.970557     5800  
1962-01-03  34.750000  34.062500  34.250000   0.968789    10200  
1962-01-04  34.312500  33.750000  33.750000   0.954646     8800  
1962-01-05  33.625000  32.625000  33.000000   0.933433    11400  
1962-01-08  33.187500  32.000000  32.750000   0.926361    17200  
...               ...        ...        ...        ...      ...  
2024-01-09  84.489998  83.669998  84.129997  84.129997  2922800  
2024-01-10  83.959999  82.489998  82.540001  82.540001  3053400  
2024-01-11  81.720001  79.919998  80.400002  80.400002  3574600  
2024-01-12  81.750000  80.580002  81.489998  81.489998  4672800  
2024-01-16  81.364998  80.769997  81.139999  81.139999   592693  

[15616 rows x 2868 columns]

(15616, 2868)

In [46]:
#De code in een functie:


def download_yahoo_data(file_path):
    """
    Downloadt gegevens van Yahoo Finance voor tickers die zijn vermeld in het opgegeven CSV-bestand.

    Parameters:
    - file_path (str): Bestandspad naar het CSV-bestand met ticker-symbolen.

    Returns:
    - pd.DataFrame: Gefilterd DataFrame met gedownloade gegevens.
    """
    df = pd.read_csv(file_path)
    symbol_list = df.iloc[:, 0].tolist()

    download = yf.download(symbol_list, group_by="ticker")
    data = download.copy()

    filtered_data = data.dropna(axis=1, how='all')
    filtered_data.columns = filtered_data.columns.remove_unused_levels()

    remaining_tickers = list(filtered_data.columns.levels[0])
    missing_tickers = list(set(symbol_list) - set(remaining_tickers))

    print("Ontbrekende tickers en het aantal:", missing_tickers, len(missing_tickers))
    display(filtered_data, filtered_data.shape)

    return filtered_data

# Voorbeeldgebruik:
# downloaded_data = download_yahoo_data('datasets/vusa_holdings.csv')


In [48]:
#Functie uitproberen
downloaded_data = download_yahoo_data('datasets/vusa_holdings.csv')

[*********************100%%**********************]  505 of 505 completed

28 Failed downloads:
['FRC', 'ANTM', 'FB', 'VIAC', 'FBHS', 'BLL', 'CERN', 'ATVI', 'DISCA', 'BRK.B', 'INFO', 'PBCT', 'RE', 'DISCK', 'SIVB', 'NLSN', 'NLOK', 'PKI', 'DRE', 'ABC', 'FISV', 'KSU', 'CTXS', 'XLNX', 'WLTW', 'TWTR']: Exception('%ticker%: No timezone found, symbol may be delisted')
['LLY']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=-1416700800&period2=1705422403&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=3pWPFvcy%2Fwq (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2548)')))"))
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1925-02-08 -> 2024-01-16)')


Ontbrekende tickers en het aantal: ['NLSN', 'FB', 'RE', 'WLTW', 'FISV', 'PBCT', 'ATVI', 'CERN', 'DISCA', 'LLY', 'ANTM', 'BRK.B', 'FBHS', 'INFO', 'DRE', 'DISCK', 'KSU', 'CTXS', 'NLOK', 'BF.B', 'TWTR', 'ABC', 'SIVB', 'VIAC', 'BLL', 'XLNX', 'FRC', 'PKI'] 28


FLT                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
1962-01-02         NaN         NaN         NaN         NaN         NaN   
1962-01-03         NaN         NaN         NaN         NaN         NaN   
1962-01-04         NaN         NaN         NaN         NaN         NaN   
1962-01-05         NaN         NaN         NaN         NaN         NaN   
1962-01-08         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2024-01-09  282.600006  285.200012  282.049988  283.690002  283.690002   
2024-01-10  283.670013  283.899994  279.820007  280.260010  280.260010   
2024-01-11  280.989990  281.320007  277.559998  279.480011  279.480011   
2024-01-12  282.070007  284.000000  280.250000  283.989990  283.989990   
2024-01-16  282.200012  284.119995  280.040009  282.359985  282.359985   

                           HSIC                                   ...  \
              Volume       Open       High        Low      Close  ...   
Date                                                              ...   
1962-01-02       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-03       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-04       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-05       NaN        NaN        NaN        NaN        NaN  ...   
1962-01-08       NaN        NaN        NaN        NaN        NaN  ...   
...              ...        ...        ...        ...        ...  ...   
2024-01-09  263900.0  75.160004  76.050003  74.870003  75.059998  ...   
2024-01-10  241500.0  74.970001  75.980003  74.580002  74.730003  ...   
2024-01-11  238200.0  74.650002  75.040001  73.220001  73.949997  ...   
2024-01-12  258800.0  74.180000  74.699997  73.559998  73.760002  ...   
2024-01-16   73658.0  73.419998  73.800003  72.400002  72.425003  ...   

                  HPQ                                         AEP             \
                  Low      Close  Adj Close     Volume       Open       High   
Date                                                                           
1962-01-02   0.124177   0.124177   0.042713  2480333.0   0.000000  35.125000   
1962-01-03   0.121516   0.122846   0.042255   507341.0   0.000000  34.750000   
1962-01-04   0.117968   0.120185   0.041340   845568.0   0.000000  34.312500   
1962-01-05   0.117525   0.117525   0.040425   338227.0   0.000000  33.625000   
1962-01-08   0.115307   0.119299   0.041035   873754.0   0.000000  33.187500   
...               ...        ...        ...        ...        ...        ...   
2024-01-09  30.139999  30.160000  30.160000  8504500.0  84.019997  84.489998   
2024-01-10  30.100000  30.340000  30.340000  8686400.0  83.870003  83.959999   
2024-01-11  29.820000  30.000000  30.000000  8835600.0  81.389999  81.720001   
2024-01-12  29.709999  29.809999  29.809999  6880900.0  80.860001  81.750000   
2024-01-16        NaN        NaN        NaN        NaN  81.000000  81.364998   

                                                      
                  Low      Close  Adj Close   Volume  
Date                                                  
1962-01-02  34.312500  34.312500   0.970557     5800  
1962-01-03  34.062500  34.250000   0.968790    10200  
1962-01-04  33.750000  33.750000   0.954647     8800  
1962-01-05  32.625000  33.000000   0.933432    11400  
1962-01-08  32.000000  32.750000   0.926360    17200  
...               ...        ...        ...      ...  
2024-01-09  83.669998  84.129997  84.129997  2922800  
2024-01-10  82.489998  82.540001  82.540001  3053400  
2024-01-11  79.919998  80.400002  80.400002  3574600  
2024-01-12  80.580002  81.489998  81.489998  4672800  
2024-01-16  80.769997  81.266899  81.266899   635133  

[15616 rows x 2862 columns]

(15616, 2862)

# ETL pipeline


# ETL Pipeline Stappenplan

## 1. Begrijp de vereisten
- Identificeer bronnen en doelopslagplaats.
- Begrijp de structuur van de doelopslagplaats.

## 2. Data Extractie (Extract)
- Selecteer bronnen (databases, API's, bestanden).
- Ontwikkel extractiescripts of -processen.

## 3. Data Transform (Transform)
- Reinig en structureer geëxtraheerde gegevens.
- Transformeer gegevens volgens doelstructuur.
- Voer bewerkingen uit zoals filtering en aggregatie.

## 4. Data Laden (Load)
- Kies een doelopslagplaats (datawarehouse, database).
- Ontwikkel scripts om gegevens te laden.

## 5. Scheduling en Automatisering
- Plan regelmatige uitvoering van ETL-processen.
- Gebruik geautomatiseerde tools of takenplanners.

## 6. Monitoring en Logging
- Implementeer monitoring voor gezondheidscontrole.
- Voeg logging toe voor probleemidentificatie.

## 7. Foutafhandeling en Herstel
- Implementeer foutafhandelingsmechanismen.
- Ontwikkel herstelmechanismen.

## 8. Documentatie
- Documenteer ETL-pipeline en processen.
- Houd documentatie up-to-date.

## 9. Beveiliging
- Implementeer beveiligingsmaatregelen.
- Beheer toegangsrechten.

## 10. Optimalisatie en Prestatietuning
- Identificeer en optimaliseer prestatieknelpunten.
- Overweeg parallelle verwerking.

## 11. Testen
- Voer uitgebreide tests uit (unit, integratie, end-to-end).
- Test met verschillende gegevenstypen.

## 12. Implementatie
- Implementeer ETL-pipeline in productieomgeving.
- Monitor initiële uitvoeringen.


# Extraheren

In [50]:
# Laden